# IQ Mixing notes

https://markimicrowave.com/technical-resources/application-notes/how-to-think-about-iq-mixers/

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import os
import sys

sys.path.insert(1, '..\\..\\Instruments\\network\\RS_Signal_Generator')
sys.path.insert(2, '..\\..\\Instruments\\Acquisition_Card')
sys.path.insert(3, '..\\..\\Instruments\\QuickSyn')
sys.path.insert(4, '..\\')

import Tuls as Ts
import RS_SMA100B
import FSL_0010
from Card_Class import PXIe5170R
import Config as par

In [13]:
QSyn_port       = 'COM35'
SG_ip           = '192.168.40.15'
Card_Name       = 'PXI1Slot3'

fsl             = FSL_0010.FSL10_synthesizer(device_address = QSyn_port)
sGen            = RS_SMA100B.SMA100B(ip = SG_ip)
pxie            = PXIe5170R(resource_name = Card_Name)

FSL_0010:	Connection successful!
SMA100B:	Connection successful!
PXIe_5170R:    Unable to establish a connection: -1074118610: The digitizer is being used by another application or process, or there is a calibration session open to the device.


In [80]:
pulse_f_min     = 1.010e9
# pulse_f_max		= 

amp_min		    = -18                                               # Set the amplitude of the signal in dBm
# amp_max			= 


sample_rate     = 250e6                                             # Maximum Value: 250.0e6
k = 4                                                               # coefficiente per prendere k*1000 punti 
																	# con la scheda, dunque devo aumentare la pulse width e period
pulse_period    = k * 1e-6                                          # 4e-6 con 250e6 dà 1000 punti
num_pts         = int(sample_rate * pulse_period)                   # min 5ns                             
pulse_delay     = 0

percent         = 5
pulse_width     = pulse_period * (1 - percent/100)                  # min 20ns


# Quick Syn
LO =  5e9
fsl_freq = round(LO, 1)
 

# MR GEN
dwc_f = 10e6                                                        # Down conversion frequency
step = 10e6
n = 20                                                              # number of steps

pulse_f_min     = LO + dwc_f
pulse_f_max		= pulse_f_min + n * step

amp_i			= -30                                               # dBm
amp_f           = -10
 
pulse_period    = 1e-6                                              # 4e-6 con 250e6 dà 1000 punti
pulse_delay     = 0
percent         = 5
pulse_width     = pulse_period * (1 - percent/100)                  # min 20ns


# PXIE
sample_rate     = 250e6                                             # Maximum Value: 250.0e6
num_pts         = int(sample_rate * pulse_period)                   # min 5ns                             
channels = {'0': 'I', 
			'1': 'Q',
			'2': None,
			'3': 'trigger'}


sGen.pul_gen_mode('SING')
sGen.pul_trig_mode('SING')
sGen.pul_gen_params(delay = pulse_delay, 
                    width = pulse_width, 
                    period = pulse_period) 


# Parameters for the Acquisition Card
pxie.coupling           = 'AC'
pxie.num_pts            = num_pts
pxie.num_records        = 1
pxie.sample_rate        = sample_rate                               # Maximum Value: 250.0e6


# FSL-0010
print(fsl.set_frequency(LO))

'Frequency set to 1 GHz.'

In [89]:
channels = {'0': 'I', 
			'1': 'Q',
			'2': None,
			'3': 'trigger'}

print(fsl.set_frequency(LO))
print(fsl.set_output('ON'))
pxie.open(trigger_channel = 3)

pota        = np.arange(amp_min, amp_min + 10 , 1)
pula        = np.arange(pulse_f_min, pulse_f_min + 0.010e9, 0.001e9)

data_dict   = {}
counter     = 1


digits_p = "{:0"+str(len(str(len(pota))))+"d}"
digits_f = "{:0"+str(len(str(len(pula))))+"d}"
for p, pot in enumerate(pota):
	data_dict[f'p{digits_p.format(p)}'] = {'power_(dBm)': pot, 'freqs': {}}
	sGen.RF_lvl_ampl(pot)

	for i, pul in enumerate(pula):
		
		sGen.RF_freq(pul) # guarda come è definito, abbiamo messo la potenza e6   
		sGen.pul_state(1)
		sGen.RF_state(1)
		# time.sleep(0.1)
		waveforms = pxie.acquisition(sGen.pul_exe_sing_trig)
		I = np.array(waveforms[I_CH].samples.tolist())
		Q = np.array(waveforms[Q_CH].samples.tolist())
		
		
		sGen.pul_state(0)
		sGen.RF_state(0)

		print(f'\r(p{digits_p.format(p)}, f{digits_f.format(i)})	: {int(counter*100/(len(pota)*len(pula)))} %', end='')
		sys.stdout.flush()
		counter += 1
		data_dict[f'p{digits_p.format(p)}']['freqs'][f'f{digits_f.format(i)}'] = {'freq_(Hz)': pul, 'I': I, 'Q': Q}
fsl.set_output('OFF')




(p09, f09)	: 100 %

'RF set to OFF'

In [90]:
# SAVE DATA ON HDF5 FILE
filename = 'Tappo_su_I_cambiocavo_e_canale.h5'
if os.path.exists(filename):
  os.remove(filename)
hdf5_write(data_dict, filename)

## NOTE
- [ ] Dai test sembra che non servano attenuazioni quando il segnale passa per il mixer, ma è necessaerio capire quanta potenza effettivamente arriva alla scheda per non friggerla.
- [ ] DOVREMO fare la CARATTERIZZAZIONE DELL'IQ-MIXER
- [ ] Studiare range di ampiezze in cui opera e magari anhe range di frequenze.
- [ ] Studiare Attenuazione che dà?
- [ ] Nel piano I-q non compare una circonferenza precisa, ma è più un'ellisse. Studiare e magari richiedere.
- [ ] Preparare i programmi, anche per l'analisi dati in loco per risparmiare tempo.
- [ ] Scrivere una manuale per la pixie, in particolare per l'uso del trigger esterno.